## Stworzenie indeksu do wszystkich zadań

In [9]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError

es = Elasticsearch("http://localhost:9200")

index_name = "fiqa_pl_corpus"
settings = {
    "settings": {
        "analysis": {
            "filter": {
                "polish_morfologik_filter": {
                    "type": "morfologik_stem"
                },
                "polish_synonym_filter": {
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                }
            },
            "analyzer": {
                "polish_analyzer_synonyms_lema": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "polish_synonym_filter",
                        "polish_morfologik_filter",
                        "lowercase"
                    ]
                },
                "polish_analyzer_synonyms_no_lema": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "polish_synonym_filter"
                    ]
                },
                "polish_analyzer_no_synonyms_lema": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "polish_morfologik_filter",
                        "lowercase"
                    ]
                },
                "polish_analyzer_no_synonyms_no_lema": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content_with_synonyms_lema": {
                "type": "text",
                "analyzer": "polish_analyzer_synonyms_lema"
            },
            "content_with_synonyms_no_lema": {
                "type": "text",
                "analyzer": "polish_analyzer_synonyms_no_lema"
            },
            "content_without_synonyms_lema": {
                "type": "text",
                "analyzer": "polish_analyzer_no_synonyms_lema"
            },
            "content_without_synonyms_no_lema": {
                "type": "text",
                "analyzer": "polish_analyzer_no_synonyms_no_lema"
            }
        }
    }
}

try:
    es.indices.create(index=index_name, body=settings)
    print(f"Index '{index_name}' created successfully.")
except RequestError as e:
    if e.error == "resource_already_exists_exception":
        print(f"Index '{index_name}' already exists.")
    else:
        print(f"Error creating index '{index_name}': {e}")


Index 'fiqa_pl_corpus' created successfully.


In [10]:
from datasets import load_dataset

fiqa_pl = load_dataset("clarin-knext/fiqa-pl", 'corpus', split='corpus')
fiqa_pl


Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})

In [18]:
corpus = fiqa_pl.to_pandas()['text']
corpus

0        Nie mówię, że nie podoba mi się też pomysł szk...
1        Tak więc nic nie zapobiega fałszywym ocenom po...
2        Nigdy nie możesz korzystać z FSA dla indywidua...
3        Samsung stworzył LCD i inne technologie płaski...
4        Oto wymagania SEC: Federalne przepisy dotycząc...
                               ...                        
57633    >Cóż, po pierwsze, drogi to coś więcej niż hob...
57634    Tak, robią. Na dotacje dla firm farmaceutyczny...
57635    >To bardzo smutne, że nie rozumiesz ludzkiej n...
57636    „Czy Twój CTO pozwolił dużej grupie użyć „„adm...
57637    Zapewnienie rządowi większej kontroli nad dyst...
Name: text, Length: 57638, dtype: object

### Dodanie danych to ES

In [19]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

def generate_actions(corpus):
    for text in corpus:
        yield {
            "_index": "fiqa_pl_corpus",
            "_source": {
                "content_with_synonyms_lema": text,
                "content_with_synonyms_no_lema": text,
                "content_without_synonyms_lema": text,
                "content_without_synonyms_no_lema": text,
            }
        }

try:
    bulk(es, generate_actions(corpus))
    print("Data loaded successfully into Elasticsearch index 'fiqa_pl_corpus'.")
except Exception as e:
    print(f"Error loading data: {e}")


Data loaded successfully into Elasticsearch index 'fiqa_pl_corpus'.


## Porównanie indeksu z synonimami i bez synonimów

In [20]:
from elasticsearch import Elasticsearch

search_term = "kwiecień"

query_with_synonyms = {
    "query": {
        "match": {
            "content_with_synonyms_lema": search_term
        }
    }
}

query_without_synonyms = {
    "query": {
        "match": {
            "content_without_synonyms_lema": search_term
        }
    }
}

try:
    response_with_synonyms = es.search(index=index_name, body=query_with_synonyms)
    count_with_synonyms = response_with_synonyms["hits"]["total"]["value"]

    response_without_synonyms = es.search(index=index_name, body=query_without_synonyms)
    count_without_synonyms = response_without_synonyms["hits"]["total"]["value"]

    print(f"Number of documents containing 'kwiecień' (including synonyms): {count_with_synonyms}")
    print(f"Number of documents containing 'kwiecień' (excluding synonyms): {count_without_synonyms}")

except Exception as e:
    print(f"Error executing queries: {e}")


Number of documents containing 'kwiecień' (including synonyms): 306
Number of documents containing 'kwiecień' (excluding synonyms): 257


In [21]:
from datasets import load_dataset

fiqa_pl_qrels = load_dataset("clarin-knext/fiqa-pl-qrels", split='test').to_pandas()
fiqa_pl_qrels

,query-id,corpus-id,score
0,8,566392,1
1,8,65404,1
2,15,325273,1
3,18,88124,1
4,26,285255,1
...,...,...,...
1701,11039,330058,1
1702,11039,91183,1
1703,11054,155053,1
1704,11054,321015,1


In [22]:
fiqa_pl_qrels.sort_values(by='query-id')

,query-id,corpus-id,score
0,8,566392,1
1,8,65404,1
2,15,325273,1
3,18,88124,1
5,26,350819,1
...,...,...,...
1702,11039,91183,1
1698,11039,293531,1
1704,11054,321015,1
1703,11054,155053,1


In [23]:
fiqa_pl_qrels[fiqa_pl_qrels['score'] != 1]

,query-id,corpus-id,score


In [24]:
fiqa_pl_qrels['query-id']

0           8
1           8
2          15
3          18
4          26
        ...  
1701    11039
1702    11039
1703    11054
1704    11054
1705    11088
Name: query-id, Length: 1706, dtype: int64

In [29]:
fiqa_pl_pd = fiqa_pl.to_pandas()
questions = load_dataset("clarin-knext/fiqa-pl", 'queries', split='queries').to_pandas()


In [31]:
import pandas as pd

questions['_id'] = questions['_id'].astype(int)
fiqa_pl_pd['_id'] = fiqa_pl_pd['_id'].astype(int)


qa_intermediate = pd.merge(questions, fiqa_pl_qrels, left_on='_id', right_on='query-id')
qa_pairs = pd.merge(qa_intermediate, fiqa_pl_pd, left_on='corpus-id', right_on='_id', suffixes=('_question', '_answer'))
qa_pairs = qa_pairs[['query-id', 'text_question', 'corpus-id', 'score', 'text_answer']]
qa_pairs

,query-id,text_question,corpus-id,score,text_answer
0,4641,Gdzie powinienem zaparkować mój fundusz na des...,44594,1,"„Po pierwsze, zazwyczaj chcesz zaparkować swój..."
1,4641,Gdzie powinienem zaparkować mój fundusz na des...,406219,1,Sugerowałbym lokalną kasę kredytową lub lokaln...
2,4641,Gdzie powinienem zaparkować mój fundusz na des...,319954,1,"Używam ING do awaryjnych oszczędności, ale w z..."
3,4641,Gdzie powinienem zaparkować mój fundusz na des...,397358,1,"To chyba dobry moment, aby zauważyć, że kredyt..."
4,4641,Gdzie powinienem zaparkować mój fundusz na des...,88327,1,"Coś z gwarancją FDIC, czyli bank. W przypadku ..."
...,...,...,...,...,...
1701,94,Wykorzystywanie punktów kart kredytowych do op...,245447,1,„Dla uproszczenia zacznijmy od rozważenia zwro...
1702,2551,Jak znaleźć tańszą alternatywę dla tradycyjnej...,413832,1,Najtańszy to jedno. Możesz absolutnie robić za...
1703,2551,Jak znaleźć tańszą alternatywę dla tradycyjnej...,450742,1,"„To, jak niskie możesz obniżyć koszty, zależy ..."
1704,2551,Jak znaleźć tańszą alternatywę dla tradycyjnej...,143100,1,Spróbuj użyć dostawcy usług VOIP lub interneto...


## Obliczenie NDCG@5

In [37]:
from datasets import load_dataset
from elasticsearch import Elasticsearch
from sklearn.metrics import ndcg_score
import numpy as np


def query_es(question, index_name, field_name):
    query = {
        "query": {
            "match": {
                field_name: question
            }
        },
        "size": 5
    }
    response = es.search(index=index_name, body=query)
    return response["hits"]["hits"]

def compute_ndcg_at_k(relevance_scores, k=5):
    scores_array = np.asarray([relevance_scores[:k]])
    ideal_array = np.ones(scores_array.shape) # scores column in qa_pairs is always 1
    return ndcg_score(ideal_array, scores_array)

configs = [
    "content_with_synonyms_lema",
    "content_with_synonyms_no_lema",
    "content_without_synonyms_lema",
    "content_without_synonyms_no_lema",
]

ndcg_scores = { config.removeprefix('content_'): [] for config in configs }

for qa_pair in qa_pairs.iterrows():
    qa_pair = qa_pair[1]
    question = qa_pair.iloc[1]

    
    for field_name in configs:
        retrieved_results = query_es(question, "fiqa_pl_corpus", field_name)
        # print(retrieved_results)
        
        retrieved_relevance = [result["_score"] for result in retrieved_results]
        
        score = compute_ndcg_at_k(retrieved_relevance, k=5)
        key = field_name.removeprefix('content_')
        ndcg_scores[key].append(score)

# Calculate average NDCG@5 for each configuration
avg_ndcg_scores = {config: np.mean(scores) for config, scores in ndcg_scores.items()}
print("Average NDCG@5 for each configuration:", avg_ndcg_scores)


Average NDCG@5 for each configuration: {'with_synonyms_lema': np.float64(1.0), 'with_synonyms_no_lema': np.float64(1.0), 'without_synonyms_lema': np.float64(1.0), 'without_synonyms_no_lema': np.float64(1.0)}


### Pytania

#### Pytanie 1: What are the strengths and weaknesses of regular expressions versus full text search regarding processing of text?


Zalety wyrażeń regularnych:
- wyrażenia regularne są prostsze od FTS
- napisanie wyrażenia regularnego jest często szybsze niż przygotowanie poprawnego indeksu w celu przeprowadzenia FTS


Wady wyrażeń regualrnych:
- Przeprowadzenie dokładnego przeszukiwania wymaga napisania dość skomplikowanego wyrażenia regularnego (np. uwzględnienie odmian słów, synonimów)
- do każdego zapytania trzeba przygotować nowe wyrażenie regularne
- każde zapytanie wymaga przeszukania całego zbioru danych, a ElasticSearch umożliwia indeksowanie i przeszukiwanie tylko indeksu

#### Pytanie 2: Can an LLM be applied in the context of searching for documents? Justify your answer, excluding the obvious observation that an LLM can be used to formulate the answer.

Teoretycznie można pointruować LLM'a aby wypisał nr dokumentu lub całą jego treść który najlepiej odppowiada na pytanie, ale byłoby to wolne, drogie i probmatyczne przez stosumkowo małą liczbę dokumentów możliwą do zmieszczenia context window LLM'a. 